# Expectation Suite

This notebook can be used to set up an expectation suite and checkpoint for running data validations using Great Expectations at runtime

#### Imports and configurations
In the cell below, packages are imported and configurations are loaded from the project_config.yml file

In [ ]:
# Imports
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
import boto3
from supporting_functions import (TestingConfiguration, 
                                  checkpoint_without_datadocs_update, 
                                  print_ge_site_link)
import os

# Load parameters from configuration file
test_config = TestingConfiguration("project_config.yml")
test_config.load_config()

#### Initialization of objects

In [ ]:
# -- 1. Initialize GE and S3 objects
s3_client = boto3.client("s3")
bucket = boto3.resource("s3").Bucket(test_config.data_bucket)
context = ge.data_context.DataContext()

#### Loading data

In [ ]:
# -- 2. Load data, generate asset name and batch identifier
df_batch = load_data()  # Needs to be defined!
batch_identifier = "BATCH IDENTIFIER OR LOGIC TO GENERATE IT GOES HERE"
asset_name = "ASSET NAME OR LOGIC TO GENERATE IT GOES HERE"

#### Generate batch request, generate suite and start validator


In [ ]:
# -- 3. Generate batch request at runtime using loaded tile
batch_request = RuntimeBatchRequest(
    datasource_name="runtime_data",
    data_connector_name="runtime_data_connector",
    data_asset_name=asset_name,
    runtime_parameters={"batch_data": df_batch},
    batch_identifiers={"batch_identifier": batch_identifier,},
)

# -- 4. Generate expectation suite, start validator
suite = context.create_expectation_suite(
    test_config.expectations_suite_name,
    overwrite_existing=True,  
)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=test_config.expectations_suite_name,
)

#### Expectations

In [ ]:
# -- 5. Set expectations
## PUT YOUR EXPECTATIONS BELOW

#### Finalize suite and create checkpoint

In [ ]:
# -- 6. Save suite
validator.save_expectation_suite(discard_failed_expectations=False)

# -- 7. Create checkpoint
#       Two options are provided here for creating a checkpoint: 
#       1. Use a SimpleCheckpoint, which contains an action to automatically update the
#          Data Docs website whenever a validation is run
#       2. Use checkpoint_without_datadocs_update, which generates the same
#          SimpleCheckpoint but without an automatic Data Docs update action. This is
#          useful if you run many validations (1000+) in parallel, since rendering the
#          Data Docs website becomes very slow in that case          

# -- 7.1 Create Simple checkpoint with automatic data docs updates (DEFAULT)
checkpoint_config = {
    "name": test_config.checkpoint_name,
    "config_version": 3,
    "class_name": "SimpleCheckpoint",
    "expectation_suite_name": test_config.expectations_suite_name,
    "run_name_template": test_config.run_name_template,
}

# -- 7.2 Create checkpoint without automatic data docs update
# checkpoint_config = checkpoint_without_datadocs_update(test_config)
context.add_checkpoint(**checkpoint_config)

#### Instantiate Data Docs website

In [ ]:
ge_site_output = context.build_data_docs()
print_ge_site_link(ge_site_output)

#### Stop Jupyter server

In [ ]:
os.system("jupyter notebook stop")